In [ ]:
import getpass
import os

In [ ]:
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

In [ ]:
os.environ["OPENAI_API_KEY"] = getpass.getpass()

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4", temperature=.9)

In [ ]:
__import__('pysqlite3')
import sys
sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')
import chromadb

In [ ]:
from langchain import hub # langchain smith contendo repositórios de prompts
from langchain_chroma import Chroma # base de dados vetorial que pode ser executada localmente
from langchain_community.document_loaders.csv_loader import CSVLoader # leitura do arquivo CSV
from langchain_core.output_parsers import StrOutputParser # responsável pela saída do modelo
from langchain_core.runnables import RunnablePassthrough # garante a integridade das entradas de dados
from langchain_openai import OpenAIEmbeddings # embeddins do gpt
from langchain_text_splitters import RecursiveCharacterTextSplitter # reparte o conteúdo em vários

In [ ]:
file_path = (
    "dataset.csv"
)

loader = CSVLoader(
    file_path=file_path,
    csv_args={
        "delimiter": ";",
        "quotechar": '"',
        "fieldnames": ["Signo", "Texto ", "Fonte"],
    },
)
data = loader.load()

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=150)
splits = text_splitter.split_documents(data)
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

In [ ]:
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")

In [ ]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [ ]:
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

rag_chain.invoke("Poderia gerar um novo Texto com base nas informações de contexto sobre o Signo de Aries?")